In [1]:
from simple_salesforce import Salesforce
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import base64

sns.set()
pd.options.display.float_format = '{:20,.2f}'.format

ModuleNotFoundError: No module named 'simple_salesforce'

## Connect to Salesforce

In [ ]:
############### CONNECT TO SALESFORCE API ###############
    
sf = Salesforce(username='', 
                password='',
                security_token='')

sf

## SOQL Query Competitor Network Sites

In [ ]:
############### LOAD COMPETITOR PRICING ###############

comp_sites_data = sf.query_all("")

comp_sites_df = pd.DataFrame(comp_sites_data['records']).drop(columns='attributes')

comp_sites_df.head()

## SOQL Query Competitor Pricing Components

In [ ]:
############### LOAD VEP ITEMS ###############

CPC_data = sf.query_all("")

CPC_df = pd.DataFrame(CPC_data['records']).drop(columns='attributes')
CPC_df.shape

In [ ]:
CPC_df.head()

## SOQL Query Buildings

In [ ]:
############### LOAD BUILDINGS ###############

bldg_data = sf.query_all("")

bldg_df = pd.DataFrame(bldg_data['records']).drop(columns='attributes')
bldg_df.head()

## SOQL Query Markets

In [ ]:
############### LOAD MARKETS ###############
mkt_data = sf.query_all("")

mkt_df = pd.DataFrame(mkt_data['records']).drop(columns='attributes')
mkt_df.head()

## Merge Markets to Buildings

In [ ]:
############### MERGE MARKETS WITH BUILDINGS ###############

bldg_df = bldg_df.merge(mkt_df, how='left', on='Market Id')

In [ ]:
bldg_df.shape

## Merge Buildings to Comp Sites

In [ ]:
############### MERGE BUILDINGS TO COMP_SITES__DF ###############

comp_sites_df = comp_sites_df.merge(bldg_df, how='inner', on='Building Id')

comp_sites_df.head()

## Create Column for Data Centers vs Enterprise

In [ ]:
############### CREATE COLUMN FOR DC vs ENTERPRISE ###############

DC = ['Carrier Hotel', 'Data Center']
comp_sites_df['DC vs Enterprise'] = np.where(comp_sites_df['Type'].isin(DC),'DC','Enterprise')

## Match Comp Accounts from Comp Sites to C_Id on Comp Pricing Components

In [ ]:
############### MATCH COMP ACCOUNTS TO COMP PRICING COMPONENTS ###############

# filter comp sites dataframe for only comp accounts and comp Ids that are in our network US&CAN buildings
temp_df = comp_sites_df[['C_Account', 'C_Id']]

# drop dups to see how many unique comp accounts we have, and create a list of unique values
temp_df = temp_df.drop_duplicates(subset='VId').reset_index(drop=True)

temp_df.head()

In [ ]:
############### DROP COMP ACCOUNTS NOT IN ON-NET US & CAN BUILDINGS ###############

# map comp accounts from our on-net US&CAN buildings to a new column, matched on comp Id
CPC_df['C_Account'] = CPC_df['C_Id'].map(temp_df.set_index('C_Id')['C_Account'])

# if there are blanks in this column, it means VEP Account is not in US&CAN on-net building, drop blank values here
CPC_df = CPC_df.dropna(axis=0, subset=['C_Account']).reset_index(drop=True)

CPC_df.shape

## Convert Currency on Comp Pricing Components to USD & Convert NRC to MRR + MAR

In [ ]:
## LOAD DICT OF CURRENCY CONVERSIONS

currencies_dict = {'AUD':1.297153,
                   'BRL':5.635906,
                   'CAD':1.256927,
                   'CHF':0.929947,
                   'DKK':6.247192,
                   'EUR':0.840278,
                   'GBP':0.721456,
                   'HKD':7.764627,
                   'JPY':108.760402,
                   'MXN':20.766766,
                   'SGD':1.342426,
                   'USD':1.000000,
                   'ZAR':14.976201
                   }

In [ ]:
## CONVERT MRC & NRC FROM COMP PRICING COMPONENTS TO USD

CPC_df['MRC (USD)'] = CPC_df['MRC'] / CPC_df['Currency'].map(currencies_dict)
CPC_df['NRC (USD)'] = CPC_df['NRC'] / CPC_df['Currency'].map(currencies_dict)

In [ ]:
## ADD COLUMN FOR MRR + MAR

CPC_df['MRR + MAR'] = CPC_df['MRC (USD)'] + (CPC_df['NRC (USD)']/CPC_df['Term'].astype(float))

CPC_df.columns # check that columns are added correctly

## Import Network Service Capabilities from CSV

In [ ]:
## Load data from the CSV

df_caps = pd.read_csv('net_service_caps.csv')

df_caps.head(20)

In [ ]:
df_caps['Pref Loc'].value_counts() # Not many locations are labelled as preferred location

In [ ]:
df_caps['Capability Type'].value_counts() # How many of each enablement level

In [ ]:
df_caps['Capability Type'].isnull().value_counts() # How many service caps don't have Capability Type listed

In [ ]:
## Drop the service caps with blank capability types

df_caps.dropna(subset = ['Capability Type'], inplace = True)

df_caps['Capability Type'].isnull().value_counts() # Only service caps with cap type listed remain

## Create column that ranks the locs by preferred then capability level

In [ ]:
############### CREATE COLUMN FOR RANKED DEMARCS ###############

df_caps['Ranked Loc'] = np.where(df_caps['Pref Loc'].str.contains('E', na = False), 1,
                                   np.where(df_caps['Capability Type'].eq('100'), 2,
                                    np.where(df_caps['Capability Type'].eq('P'), 3,
                                     np.where(df_caps['Capability Type'].eq('Pass'), 4,
                                      np.where(df_caps['Capability Type'].eq('Ex'), 5,
                                       np.where(df_caps['Capability Type'].eq('Path'), 6,
                                        np.where(df_caps['Capability Type'].eq('Enabled'), 7,
                                         np.where(df_caps['Capability Type'].eq('Accessible'), 8,
                                                  'No Rank'
                                                 ))))))))

df_caps.head()

In [ ]:
df_caps['Ranked Loc'].value_counts() # How many of each rank do we have?

In [ ]:
## Drop non-ranked loc values

df_caps.drop(df_caps.index[df_caps['Ranked Loc']=='No Rank'], inplace = True)

df_caps.head(10)

## Check for ranked location ties in each building

In [ ]:
## TEST FOR DUPLICATES WHERE TIE BREAKERS MAY BE NEEDED

df_caps.sort_values(['Building Id', 'Ranked Loc'], ascending=[False, True], inplace = True)

## Check if there are duplicate values for building and ranked loc. 
## This would indicate a need for a tie breaker.

dups_df = df_caps[df_caps.duplicated(subset = ['Building Id', 'Ranked Loc'], 
                                     keep = False)]

dups_df.tail()

## Add tie breaker for each building to determine which ranked loc to use

In [ ]:
df_caps.sort_values(['Building Id', 'Ranked Loc', 'Specific Building Rank'], 
                    ascending=[False, True, True], inplace = True)

## Keep only the lowest Network Specific Building Rank for the Building
df_caps.drop_duplicates(subset = ['Building Id'], keep = 'first', inplace = True)

df_caps.head()

## Load in Pricing

In [ ]:
############### PRICING FOR ONE LOCATION ###############

'''
Listed below are the term multipliers and building multipliers.
'''

#term multipliers:
mrc_12 = 1.35
mrc_24 = 1.15
mrc_36 = 1.00
mrc_48 = 0.85
mrc_60 = 0.65


bldg_multipliers = {

1 : 0,
2 : 0.80,
3 : 0.85,
4 : 0.90,
5 : 0.95,
6 : 0.96,
7 : 0.97,
8 : 0.98,
9: 0.99,
10 : 1.00,
11 : 1.01,
12 : 1.02,
13 : 1.03,
14 : 1.04,
15 : 1.05,
16 : 1.80,
17 : 1.90,
18 : 1.91,
19 : 1.92,
20 : 2.00,
21 : 2.15,
22 : 2.25,
23 : 2.35,
24 : 2.45,
25 : 2.55
}

In [ ]:
############### LOAD BASE PRICING FROM CSV ###############

df_pricing = pd.read_csv('file.csv', index_col = 'BW')

df_pricing.head(20)

In [ ]:
## Calculate MRC by term

df_pricing['12mo MRC'] = (df_pricing['MRC'] * mrc_12)
df_pricing['24mo MRC'] = (df_pricing['MRC'] * mrc_24)
df_pricing['36mo MRC'] = (df_pricing['MRC'] * mrc_36)
df_pricing['48mo MRC'] = (df_pricing['MRC'] * mrc_48)
df_pricing['60mo MRC'] = (df_pricing['MRC'] * mrc_60)

df_pricing.head(50)

## Map building multipliers to service caps based on Specific Building Rank

In [ ]:
## Map building multipliers to service caps

df_caps['Building Multiplier'] = df_caps['Specific Building Rank'].map(bldg_multipliers)

df_caps.head()

## Merge building multipliers from service caps to comp sites

In [ ]:
## Merge building multiplier from service caps to psites
comp_sites_df = comp_sites_df.merge(df_caps[['Building Id', 'Building Multiplier']], how = 'left', on = 'Building Id' )

# Null values for Building Multiplier are buildings that are not enabled, drop these
comp_sites_df.dropna(subset = ['Building Multiplier'], inplace = True)

ps_df.head()

# Analyze & Visualize Data

In [ ]:
#CPC_df['Term'].value_counts()
#CPC_df['Handoff'].value_counts()
#CPC_df['Bandwidth'].value_counts()
#CPC_df['C_Account'].value_counts()
#CPC_df['Tier'].value_counts()

## Use this block for quick viewing of pricing info from Comp Pricing Components

In [ ]:
## FOR EASY VIEWING: USE THIS TOOL TO PULL SPECIFIC PRICING FROM COMP PRICING COMPONENTS

term_ez = '24'
handoff_ez = '100G'
bw_ez = '500Mb'

test_df = CPC_df[(CPC_df['Term'] == term_ez) 
                 & (CPC_df['Handoff'] == handoff_ez) 
                 & (CPC_df['Bandwidth'] == bw_ez)]

ave_price = test_df['MRR + MAR'].mean()
print(f'Average Price across Comp Accounts: ${ave_price:,.2f}')

## Use this block to get quick summary of outliers in Comp Sites pricing

In [ ]:
############################ TESTING ###################################
## This is the same filter and merge as the function to pull pricing, use this to get test views of data

test_comp_df = comp_sites_df

## Select term, handoff, and bandwidth below
test_view = CPC_df[(CPC_df['Term'] == '24') 
                    & (CPC_df['Handoff'] == '100G') 
                    & (CPC_df['Bandwidth'] == '1G')]

test_pricing = test_comp_df.merge(test_view[['C_Id', 'Tier', 'Term', 'MRR + MAR', 'MRC', 'NRC']], 
                            how = 'inner', left_on = ['C_Id','Pricing Tier'], 
                            right_on = ['C_Id', 'Tier']).drop(columns='Tier')

market_test = '' # if you want to slice by market, select market here

## Use these views to check max values and other filters

test_pricing = test_pricing[test_pricing['Market Name'] == market_test] # to slice by market, uncomment this line
#test_pricing[test_pricing['DC vs Enterprise'] == 'DC'].head(10) # to slice by DC vs Enterprise, uncomment this line

test_pricing.loc[test_pricing['MRR + MAR'].idxmax()] # views max MRR + MAR in dataset
#test_pricing.loc[test_pricing['MRC'].idxmax()] # views max MRC in dataset
#test_pricing.loc[test_pricing['NRC'].idxmax()] # views max NRC in dataset

test_pricing.loc[test_pricing['MRR + MAR'].idxmin()] # views min MRR + MAR in dataset
#test_pricing.loc[test_pricing['MRC'].idxmin()] # views min MRC in dataset
#test_pricing.loc[test_pricing['NRC'].idxmin()] # views min NRC in dataset

#test_pricing.head()

## Create function to print summary of pricing stats

In [ ]:
#### Bandwidth / Handoff pairs:

def get_avg_pricing (term, handoff, bw, DCE, market = None):
    # create copy of comp sites dataframe
    temp_comp_df = comp_sites_df

    # filter CPC dataframe to only term, handoff, and bw we want to get pricing for
    easy_view = CPC_df[(CPC_df['Term'] == term) 
                        & (CPC_df['Handoff'] == handoff) 
                        & (CPC_df['Bandwidth'] == bw)]
    
    # merge filtered CPC_df to comp sites to see pricing for selected term, handoff, bw
    pricing = temp_comp_df.merge(easy_view[['C_Id', 'Tier', 'MRR + MAR']], 
                            how = 'inner', left_on = ['C_Id','Pricing Tier'], 
                            right_on = ['C_Id', 'Tier']).drop(columns='Tier')
    
    # grab MRC and NRC from base pricing for this bandwidth, multiply by building multiplier and convert to MRR + MAR
    base_MRC = df_pricing.loc[bw][f'{term}mo MRC']
    base_NRC = df_pricing.loc[bw]['NRC']
    pricing['Price_MRR + MAR'] = (pricing['Building Multiplier'] * base_MRC) + (base_NRC / int(term)) 
    
    if market != None:
        pricing = pricing[pricing['Market Name'] == market]
    
    pricing = pricing[pricing['DC vs Enterprise'] == DCE]
    
    # create outputs
    ave_price = pricing['MRR + MAR'].mean()
    min_price = pricing['MRR + MAR'].min()
    max_price = pricing['MRR + MAR'].max()
    med_price = pricing['MRR + MAR'].median()
    count_of_sites = pricing['Comp Site Name'].nunique()
    company_ave = pricing['Price_MRR + MAR'].mean()
    diff_ave = company_ave - ave_price
    
    if market != None:
        print(f'''{term}mo, {handoff}, {bw}, {DCE}, {market}: 
        avg = ${ave_price:,.2f}
        median = ${med_price:,.2f}
        max = ${max_price:,.2f}
        min = ${min_price:,.2f} 
        total_sites = {count_of_sites}
        avg (single loc) = ${company_ave:,.2f}
        price difference = ${diff_ave:,.2f}''')
    else:
        print(f'''{term}mo, {handoff}, {bw}, {DCE}: 
        avg = ${ave_price:,.2f}
        median = ${med_price:,.2f}
        max = ${max_price:,.2f}
        min = ${min_price:,.2f} 
        total_sites = {count_of_sites}
        avg (single loc) = ${company_ave:,.2f}
        price difference = ${diff_ave:,.2f}''')
    
    return 


## Create function to generate boxplot for pricing stats

In [ ]:
def box_plot_pricing (term, handoff, bw, DCE, market = None):
    # create copy of comp sites dataframe
    temp1_comp_df = comp_sites_df

    # filter CPC dataframe to only term, handoff, and bw we want to get pricing for
    easy_view = CPC_df[(CPC_df['Term'] == term) 
                        & (CPC_df['Handoff'] == handoff) 
                        & (CPC_df['Bandwidth'] == bw)]
    
    # merge filtered CPC df to comp sites to see pricing for selected term, handoff, bw
    pricing = temp1_comp_df.merge(easy_view[['C_Id', 'Tier', 'MRR + MAR']], 
                            how = 'inner', left_on = ['C_Id','Pricing Tier'], 
                            right_on = ['C_Id', 'Tier']).drop(columns='Tier')
    
    # grab MRC and NRC from pricing for this bandwidth, multiply by building multiplier and convert to MRR + MAR
    base_MRC = df_pricing.loc[bw][f'{term}mo MRC']
    base_NRC = df_pricing.loc[bw]['NRC']
    pricing['MRR + MAR'] = (pricing['Building Multiplier'] * base_MRC) + (base_NRC / int(term)) 
    
    if market != None:
        pricing = pricing[pricing['Market Name'] == market]
    
    pricing = pricing[pricing['DC vs Enterprise'] == DCE]
    
    # create outputs
    ave_price = pricing['MRR + MAR'].mean()
    min_price = pricing['MRR + MAR'].min()
    max_price = pricing['MRR + MAR'].max()
    med_price = pricing['MRR + MAR'].median()
    count_of_sites = pricing['Partner Site Name'].nunique()
    _ave = pricing['MRR + MAR'].mean()
    diff_ave = _ave - ave_price
    
    # Make box plot object
    fig, axes = plt.subplots(1,2, figsize=(8,4))
 
    # Plot competitor data
    axes[0].boxplot(pricing['MRR + MAR'], showmeans=True)
    axes[0].set_title('Competitors')
    axes[0].set_ylabel('Price (USD)')
    axes[0].set_xlabel('')
    
    # Plot base data
    axes[1].boxplot(pricing['MRR + MAR'], showmeans=True)
    axes[1].set_title('(Sinle Loc)')
    axes[1].set_xlabel('')
    
    # Set Overall Title
    if market != None:
        fig.suptitle(f'{bw} - {DCE}\n', size = 14)
    else:
        fig.suptitle(f'Pricing for {term}mo, {bw}, {DCE}\n', size = 14)
    
    fig.subplots_adjust(wspace=0.25, top=0.85)
    
    # show plot
    plt.show()
    
    return 

## Create function to generate scatter plot for pricing stats

In [ ]:
def scatter_plot_pricing (term, handoff, bw, market = None):
    # Create copy of PS dataframe
    temp2_comp_df = comp_sites_df

    # Filter CPC dataframe to only term, handoff, and bw we want to get pricing for
    easy_view = CPC_df[(CPC_df['Term'] == term) 
                        & (CPC_df['Handoff'] == handoff) 
                        & (CPC_df['Bandwidth'] == bw)]
    
    # Merge filtered CPC df to comp sites to see pricing for selected term, handoff, bw
    pricing = temp2_comp_df.merge(easy_view[['C_Id', 'Tier', 'MRR + MAR']], 
                            how = 'inner', left_on = ['C_Id','Pricing Tier'], 
                            right_on = ['C_Id', 'Tier']).drop(columns='Tier')
    
    # Grab MRC and NRC from base pricing for this bandwidth, multiply by building multiplier and convert to MRR + MAR
    base_MRC = df_pricing.loc[bw][f'{term}mo MRC']
    base_NRC = df_pricing.loc[bw]['NRC']
    pricing['MRR + MAR'] = (pricing['Building Multiplier'] * base_MRC) + (base_NRC / int(term)) 
    
    if market != None:
        pricing = pricing[pricing['Market Name'] == market]
    
    # Create outputs
    ave_price = pricing['MRR + MAR'].mean()
    min_price = pricing['MRR + MAR'].min()
    max_price = pricing['MRR + MAR'].max()
    med_price = pricing['MRR + MAR'].median()
    count_of_sites = pricing['Partner Site Name'].nunique()
    _ave = pricing['Base MRR + MAR'].mean()
    diff_ave = _ave - ave_price
    _dc_ave = pricing[pricing['DC vs Enterprise'] == 'DC']['MRR + MAR'].mean()
    _enterprise_ave = pricing[pricing['DC vs Enterprise'] == 'Enterprise']['MRR + MAR'].mean()
    
    ##########
    
    # Group Avg Price by C_Account
    grouped_df = pricing.groupby('C_Account')['MRR + MAR'].mean().reset_index()
    
    dc_df = pricing[pricing['DC vs Enterprise'] == 'DC']
    dc_grouped = dc_df.groupby('C_Account')['MRR + MAR'].mean().reset_index()
    
    enterprise_df = pricing[pricing['DC vs Enterprise'] == 'Enterprise']
    enterprise_grouped = enterprise_df.groupby('C_Account')['MRR + MAR'].mean().reset_index()
    
    # Rename columns
    grouped_df.columns = ['Account', 'Overall Avg Pricing']
    dc_grouped.columns = ['Account', 'Avg DC Pricing']
    enterprise_grouped.columns = ['Account', 'Avg Enterprise Pricing']
    
    # Merge groupbys
    grouped_df = grouped_df.merge(dc_grouped, how = 'left', on = 'Account')
    grouped_df = grouped_df.merge(enterprise_grouped, how = 'left', on = 'Account')
    
    # Append Base Avg Price to grouped
    to_add = {'Account': '(Single Loc)', 
              'Overall Avg Pricing': _ave,
              'Avg DC Pricing': _dc_ave,
              'Avg Enterprise Pricing': _enterprise_ave}
    
    grouped_df = grouped_df.append(to_add, ignore_index = True).sort_values(by = 'Overall Avg Pricing')
    colors = np.where(grouped_df['Account'].eq('(Single Loc)'), 'orangered', 'royalblue')
    
    # Make scatter plot object
    x_dc = grouped_df['Avg DC Pricing']
    x_enterprise = grouped_df['Avg Enterprise Pricing']
    y = grouped_df['Account']

    fig = plt.scatter(x_dc, y, c = colors, marker = '^', label = 'DC', alpha = 0.8)
    fig = plt.scatter(x_enterprise, y, c = colors, label = 'Enterprise', alpha = 0.5)

    # Set Overall Title
    plt.title(f'{bw}\n', size = 14)
    plt.xlabel('\nAvg Price (USD)')
    plt.legend(loc='lower right')
                           
    # Show plot
    plt.show()
    
    return

## Select your pricing combination to print summary of pricing stats:

In [ ]:
############################ CALL GET AVG PRICING FUNCTION HERE ###################################

# choose your term, DC vs Enterprise, bandwidth, market here
term_var = '36' 
DCE_var = 'DC'
bw_var = '100mb'
market_var = '' # If not slicing by market, comment this out and change to market = None below

# run function for selected combinations to get pricing
get_avg_pricing(term_var,'100G', bw_var, DCE_var, market = market_var)

## Run scatter plot function here:

In [ ]:
############################ CALL SCATTERPLOT PRICING FUNCTION HERE ###################################

# Uses same term, DCE, bandwidth, and market variables from above block and shows scatter plot

scatter_plot_pricing(term_var,'100G', bw_var, market = market_var)

## Select pricing combination for Box Plot here:

In [ ]:
############################ CALL BOXPLOT PRICING FUNCTION HERE ###################################

# Uses same term, DCE, bandwidth, and market variables from above block and shows box plot

box_plot_pricing(term_var,'100G', bw_var, DCE_var, market = market_var)